# Librerias

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    accuracy_score, 
    classification_report, 
    confusion_matrix
)
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from category_encoders import TargetEncoder

# Datos

In [7]:
df=pd.read_csv(r'C:\Users\Usuario\Documents\Trabajo Ingenieria Modelos\Bases de Datos\train.csv', encoding="utf-8")
df.head()

,ID,PERIODO_ACADEMICO,E_PRGM_ACADEMICO,E_PRGM_DEPARTAMENTO,E_VALORMATRICULAUNIVERSIDAD,E_HORASSEMANATRABAJA,F_ESTRATOVIVIENDA,F_TIENEINTERNET,F_EDUCACIONPADRE,F_TIENELAVADORA,...,E_PRIVADO_LIBERTAD,E_PAGOMATRICULAPROPIO,F_TIENECOMPUTADOR,F_TIENEINTERNET.1,F_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,INDICADOR_1,INDICADOR_2,INDICADOR_3,INDICADOR_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


# Preprocesamiento General

In [ ]:
if df['F_TIENEINTERNET'].equals(df['F_TIENEINTERNET.1']):
    df = df.drop(columns=['F_TIENEINTERNET.1'])
    print("Se elimino la variable F_TIENEINTERNET.1")

# Moda global para variables categóricas simples
variables_imputar = ['F_ESTRATOVIVIENDA', 'F_EDUCACIONMADRE', 'F_EDUCACIONPADRE']
for col in variables_imputar:
    print(f"\n📊 Variable: {col}")
    nulos_antes = df[col].isna().sum()
    print(f" - Valores nulos antes de imputar: {nulos_antes}")
    moda_valor = df[col].mode()[0]
    print(f" - Valor usado para imputar (moda): {moda_valor}")
    df.loc[:, col] = df[col].fillna(moda_valor)
    nulos_despues = df[col].isna().sum()
    print(f" - Valores nulos después de imputar: {nulos_despues}")
print("\n✅ Imputación completada correctamente.")

# Imputación por grupos para variables categóricas
imputaciones_grupales = {
    'F_TIENEAUTOMOVIL': 'F_ESTRATOVIVIENDA',
    'F_TIENELAVADORA': 'F_ESTRATOVIVIENDA',
    'F_TIENEINTERNET': 'F_ESTRATOVIVIENDA',
    'F_TIENECOMPUTADOR': 'F_TIENEINTERNET',
    'E_PAGOMATRICULAPROPIO': 'F_ESTRATOVIVIENDA',
    'E_HORASSEMANATRABAJA': 'F_ESTRATOVIVIENDA',
    'E_VALORMATRICULAUNIVERSIDAD': 'F_ESTRATOVIVIENDA'
}

for var, grupo in imputaciones_grupales.items():
    print(f"\n📊 Variable: {var} (agrupada por {grupo})")
    nulos_antes = df[var].isna().sum()
    print(f" - Valores nulos antes de imputar: {nulos_antes}")
    
    # Imputación por moda dentro del grupo
    df[var] = df.groupby(grupo)[var].transform(
        lambda x: x.fillna(x.mode()[0] if not x.mode().empty else 'No')
    )
    
    nulos_despues = df[var].isna().sum()
    print(f" - Valores nulos después de imputar: {nulos_despues}")

print("\n✅ Imputación por grupos completada correctamente.")

# Convertir PERIODO_ACADEMICO a categórica
df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype(str)
df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype("category")

def quitar_tildes(texto):
    if isinstance(texto, str):
        texto = unicodedata.normalize("NFD", texto)
        texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")
        return texto.upper()  
    return texto

df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(quitar_tildes)
print("Homogeneización Realizada")

Se elimino la variable F_TIENEINTERNET.1


In [19]:
import pandas as pd
import unicodedata

def preprocesar_datos(df):
    df = df.copy()

    # -------------------------------------
    # 1. Eliminar duplicado F_TIENEINTERNET.1
    # -------------------------------------
    if "F_TIENEINTERNET.1" in df.columns:
        if df["F_TIENEINTERNET"].equals(df["F_TIENEINTERNET.1"]):
            df = df.drop(columns=["F_TIENEINTERNET.1"])
            print("🧹 Se eliminó la variable duplicada: F_TIENEINTERNET.1")

    # -------------------------------------
    # 2. Imputación de moda global
    # -------------------------------------
    variables_imputar = [
        "F_ESTRATOVIVIENDA",
        "F_EDUCACIONMADRE",
        "F_EDUCACIONPADRE"
    ]

    for col in variables_imputar:
        if col in df.columns:
            moda = df[col].mode()[0]
            df[col] = df[col].fillna(moda)

    print("📌 Imputación global completada")

    # -------------------------------------
    # 3. Imputación por grupos (estrato, internet)
    # -------------------------------------
    imputaciones_grupales = {
        'F_TIENEAUTOMOVIL': 'F_ESTRATOVIVIENDA',
        'F_TIENELAVADORA': 'F_ESTRATOVIVIENDA',
        'F_TIENEINTERNET': 'F_ESTRATOVIVIENDA',
        'F_TIENECOMPUTADOR': 'F_TIENEINTERNET',
        'E_PAGOMATRICULAPROPIO': 'F_ESTRATOVIVIENDA',
        'E_HORASSEMANATRABAJA': 'F_ESTRATOVIVIENDA',
        'E_VALORMATRICULAUNIVERSIDAD': 'F_ESTRATOVIVIENDA'
    }

    for col, grupo in imputaciones_grupales.items():
        if col in df.columns and grupo in df.columns:

            df[col] = df.groupby(grupo)[col].transform(
                lambda x: x.fillna(x.mode()[0] if not x.mode().empty else "No")
            )

    print("📌 Imputación por grupos completada")

    # -------------------------------------
    # 4. Convertir PERIODO_ACADEMICO a categoría
    # -------------------------------------
    if "PERIODO_ACADEMICO" in df.columns:
        df["PERIODO_ACADEMICO"] = df["PERIODO_ACADEMICO"].astype(str).astype("category")

    # -------------------------------------
    # 5. Función para quitar tildes
    # -------------------------------------
    def quitar_tildes(texto):
        if isinstance(texto, str):
            texto = unicodedata.normalize("NFD", texto)
            texto = "".join(c for c in texto if unicodedata.category(c) != "Mn")
            return texto.upper()
        return texto

    # Aplicar a PRGM_ACADEMICO si existe
    if "E_PRGM_ACADEMICO" in df.columns:
        df["E_PRGM_ACADEMICO"] = df["E_PRGM_ACADEMICO"].apply(quitar_tildes)

    print("✨ Homogeneización de textos realizada")
    print("✅ Preprocesamiento completado")

    return df


# CaBoost

In [17]:
from catboost import CatBoostClassifier, Pool
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# ===========================
# 1. DEFINIR VARIABLES
# ===========================

y = df["RENDIMIENTO_GLOBAL"]

cat_cols = [
    'F_ESTRATOVIVIENDA',
    'F_EDUCACIONMADRE',
    'F_EDUCACIONPADRE',
    'F_TIENEINTERNET',
    'F_TIENELAVADORA',
    'F_TIENEAUTOMOVIL',
    'F_TIENECOMPUTADOR',
    'E_PRIVADO_LIBERTAD',
    'E_PAGOMATRICULAPROPIO',
    'E_HORASSEMANATRABAJA'
]

num_cols = [
    'INDICADOR_1', 'INDICADOR_2', 'INDICADOR_3', 'INDICADOR_4'
]

X = df[cat_cols + num_cols]

# ===========================
# 2. SPLIT TRAIN / TEST
# ===========================

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.25, random_state=42, stratify=y
)

# ===========================
# 3. POOLS PARA CATBOOST
# ===========================

train_pool = Pool(X_train, y_train, cat_features=cat_cols)
test_pool = Pool(X_test, y_test, cat_features=cat_cols)

# ===========================
# 4. MODELO CATBOOST
# ===========================

cat_model = CatBoostClassifier(
    iterations=600,
    depth=8,
    learning_rate=0.05,
    loss_function='MultiClass',
    eval_metric='Accuracy',
    random_seed=42,
    verbose=150
)

# ===========================
# 5. ENTRENAR
# ===========================

cat_model.fit(train_pool)

# ===========================
# 6. PREDICT & METRICS
# ===========================

pred_cat = cat_model.predict(test_pool)

print("\n=== CATBOOST MODEL ===")
print(classification_report(y_test, pred_cat))
print(confusion_matrix(y_test, pred_cat))


0:	learn: 0.3598787	total: 2.44s	remaining: 24m 19s
150:	learn: 0.3747023	total: 7m 25s	remaining: 22m 3s
300:	learn: 0.3789362	total: 15m 3s	remaining: 14m 57s
450:	learn: 0.3847278	total: 23m 16s	remaining: 7m 41s
599:	learn: 0.3900207	total: 31m 42s	remaining: 0us

=== CATBOOST MODEL ===
              precision    recall  f1-score   support

        alto       0.44      0.56      0.49     43905
        bajo       0.38      0.49      0.43     43246
  medio-alto       0.29      0.17      0.21     42905
  medio-bajo       0.30      0.25      0.27     43069

    accuracy                           0.37    173125
   macro avg       0.35      0.37      0.35    173125
weighted avg       0.35      0.37      0.35    173125

[[24677  7110  6170  5948]
 [ 7280 21197  5261  9508]
 [14573 11449  7274  9609]
 [10002 15542  6752 10773]]
